In [1]:
from agents import WebSearchTool, trace, Agent, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import Markdown, display

In [2]:
load_dotenv(override=True)

True

In [3]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

In [6]:
model = "gpt-4o-mini"
web_model = "gpt-5-nano"

In [ ]:
search_agent = Agent(
    name="Search Agent",
    model=web_model,
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model_settings=ModelSettings(tool_choice="required")
)

In [10]:
message = "Latest AI frameworks in 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

    display(Markdown(result.final_output))

Agent-centric AI frameworks solidify 2025. Pocketflow emerges as a Python framework for Human-AI co-design, with flow-based orchestration, modular Nodes, Flow-as-Node, and explicit control to manage complex agent workflows. LightAgent, a lightweight open-source framework, adds memory, tools, and Tree-of-Thought, with a 0.4.0 release in June 2025 and MCP protocol integration for cross-model tool use. ([arxiv.org](https://arxiv.org/abs/2504.03771?utm_source=openai))

Beyond orchestration, RL-driven and testing frameworks proliferate. Agent Lightning presents a flexible approach to train any AI agent with reinforcement learning, decoupling agent execution from training to work with existing stacks (e.g., LangChain, AutoGen); published Aug 2025. DoomArena provides a plug-in framework to stress-test AI agents against evolving security threats, enabling configurable threat models. PiFlow proposes a principled, information-theoretic multi-agent framework for automated scientific discovery. ([arxiv.org](https://arxiv.org/abs/2508.03680?utm_source=openai))

Deployment/inference frameworks advance as well. Intel’s OpenVINO 2025.3 (Sept 3, 2025) expands GenAI coverage, adds model support, and boosts NPU integration for efficient edge/cloud/local deployments. This release tightens model conversion, hardware acceleration, and pipeline capabilities for agentic workflows. ([docs.openvino.ai](https://docs.openvino.ai/2025/about-openvino/release-notes-openvino.html?utm_source=openai))

In [11]:
HOW_MANY_SEARCHES = 2

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

class WebSearchItem(BaseModel):
    reason:str
    "Your reasoning for why this search term is important to the query"

    query:str
    "The search term to use for the web search"

class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem]
    """A list of web searches to perform to best answer the query"""

In [ ]:
planner_agent = Agent(
    name="Planner Agent",
    instructions=INSTRUCTIONS
)